# Assignment 2

## Part 1

In this part a prediction error method will be used to estimate the parameters of a the data generated in part 1 in exercise 1. 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from scipy.optimize import minimize
data = pd.read_csv('/Users/loveredin/Documents/GitHub/advanced_time_series_analysis/assignment_2/data.csv', index_col = 0)
print(data.head())
X_t = np.array(data['Value'])
print(X_t)

      Value
0  0.000000
1  2.496714
2  2.361079
3  3.119904
4  4.147011
[ 0.          2.49671415  2.36107853  3.11990424  4.14701071  2.59524877
  2.2849128   4.03619537  3.5746738   2.24546037  2.99165212  2.13491273
  1.96125279  2.63421283  0.61356232  0.39779463  1.5172714   1.29062316
  2.57237196  1.60645032  0.90898636  3.64744604  2.50371291  2.56827079
  1.08890597  1.67339847  2.44560228  1.33812688  2.64332339  1.92802599
  2.09391145  1.81707568  4.21569332  2.82964144  1.50821736  3.12418838
  1.40399403  2.4896624   0.53826236  0.77946642  2.35275452  3.20901748
  2.81317178  2.44698607  2.18829352  0.95913671  1.47198313  1.83375786
  3.4238738   3.02839305  0.84263845  2.49261166  2.11344005  1.74576601
  2.96082949  3.62316542  3.6559132   1.89196512  2.06918065  2.74509956
  3.52456504  2.22573877  2.25948878  1.34556278  1.07290593  3.02710701
  3.96166143  2.72032216  3.54759733  3.07115549  1.96911134  2.75521787
  4.08908014  2.78198999  4.12104165  0.20446323  2.

In [ ]:
def kernel(x, kernel_function = 'gaussian'):

    # Define different cases of kernel functions
    if kernel_function == 'gaussian':
        return np.exp(-0.5 * x ** 2) / np.sqrt(2 * np.pi)
    elif kernel_function == 'tricube':
        in_interval = np.abs(x) <= 1
        weights = np.zeros_like(x, dtype = float)
        weights[in_interval] = (1 - np.abs(x[in_interval]) ** 3) ** 3
        return weights
    else:
        return 1 / 2 * ((x >= -1) & (x <= 1)).astype(float)


In [7]:
# Algo:
# Define the loss function
# Taylor expansion of loss function, done analytically
# Evaluate the taylor expansion of order 2, in the point theta_0
# Calculate gradient and hessian, evaluated at theta_0 and theta_est
# Use the scipy function minimization, when defined the loss function, to estimate the new parameters
# Estimate new parameters based on old, equation

# Only make small adjustments, 

# The data we are working with and am trying to estimate the parameters for is a setar model, with four parameters, theta_00, theta_01, theta_10, theta, 
# Want to compute the value for all N time values of the time series


def conditional_expectation(X_t_lagged, theta, threshold = 0):
    return np.where(X_t_lagged < 0, theta[0][0] + theta[0][1] * X_t_lagged, theta[1][0] + theta[1][1] * X_t_lagged[-1])
    
def loss_function(X_t, theta, conditional_expectation):
    # This function defines the lag for which, time series to use, in this case it is going to be the lagged 1 time series
    return sum((X_t[1 : ] - conditional_expectation(X_t[ : -1], theta)) ** 2)   

theta_0 = np.zeros([2, 2])

theta_hat = minimize(lambda theta: loss_function(X_t, theta, conditional_expectation), theta_0)

 

ValueError: 'x0' must only have one dimension.